In [ ]:
using Flux, Statistics
using Flux: crossentropy, throttle, logitcrossentropy
using Base.Iterators: repeated, partition, flatten
using DelimitedFiles
using LinearAlgebra
using GLM
using RDatasets
using BSON: @save, @load
using Random
using Base.Threads 
using CUDA

function runNNfiles(args)
    runNumber = parse(Int64, args[1])
    algorithm = args[2]
    angles = parse(Int64, args[3])
    folder = "/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/edep_extraction_data/processed_tumors_28_28_28"
    fileKey = "processed_edep_"
    validationSize = 1.05
    
    #CUDA.memory_status()
    
    lightValue = 128
    contrast = x -> for i = 1:size(x)[1] 
           for r = 1:size(x)[2]
               if x[i,r] > lightValue
                   x[i,r] = (sqrt(x[i,r]/lightValue) * sqrt(255/lightValue)) * lightValue
               else
                   x[i,r] = (x[i,r]/lightValue)^2*lightValue
               end
           end
       end


    cd(folder) #Move directory to directed folder

    pixelNumber = size(split.(Iterators.take(eachline(filter(x->occursin(fileKey, x), readdir())[1]), 1), '\t')[1])[1]
    fileNumber = size(filter(x->occursin(fileKey, x), readdir()))[1] - 1

    print(string("pixels: ", pixelNumber, '\n', "fileNumber: ", fileNumber, '\n'))
     
    dataArrayCPU = (Array{Float32}(undef, Int16(sqrt(pixelNumber)), Int16(sqrt(pixelNumber)), angles, 1+(Int16(div(fileNumber, validationSize)))))
    targetArrayCPU = (Array{Float32}(undef, Int16(784^(3/2)), 1+(Int16(div(fileNumber, validationSize)))))
    #targetArrayCPU = (Array{Float32}(undef, Int32(28*28*angles), (Int16(div(fileNumber, validationSize)))))
    validationArrayCPU = (Array{Float32}(undef, Int16(sqrt(pixelNumber)), Int16(sqrt(pixelNumber)), angles, (fileNumber) - Int16(div((fileNumber), validationSize))))
    validationTargetArrayCPU = (Array{Float32}(undef, Int16(784^(3/2)), (fileNumber) - Int16(div((fileNumber), validationSize))))
    #validationTargetArrayCPU = (Array{Float32}(undef, Int32(28*28*angles), (fileNumber) - Int16(div((fileNumber), validationSize))))
    testArrayCPU = (Array{Float32}(undef, Int16(sqrt(pixelNumber)), Int16(sqrt(pixelNumber)), Int16(angles), 1))
    testTargetArrayCPU = (Array{Float32}(undef, Int16(784^(3/2)), 1))
    #testTargetArrayCPU = (Array{Float32}(undef, Int32(28*28*angles), 1))
    
    @show typeof(dataArrayCPU)

    for i = 1:angles
        maxVal = maximum(readdlm(filter(x->occursin("processed", x), readdir())[1]))
        minVal = minimum(readdlm(filter(x->occursin("processed", x), readdir())[1]))
        temp = (transpose(reshape(readdlm(filter(x->occursin("processed", x), readdir())[1])[Int64(i*60/angles),:], (28,28)).-minVal)*255/maxVal)
        contrast(temp)
        testArrayCPU[:,:,i,1] = temp
    end
    
    edepNumber = filter(x->occursin(fileKey, x), readdir())[1][16:18]
    tumorNumber = filter(x->occursin("tumor", x), readdir())[1][16:18]
    maxVal = maximum(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")))
    minVal = minimum(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")))
    #testTargetArrayCPU[:,1] = (transpose(reshape(testArrayCPU[:,:,:,1], (28*28*angles, 1)))) 
    testTargetArrayCPU[:,1] = ((transpose(reshape(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")), (21952,1)).-minVal)*255/maxVal))
    print("Test data: ", string("processed_edep_", edepNumber, ".txt"), "/", string("tumor_28_28_28_", edepNumber, "_montage.txt"), '\n')


    for r = 1:Int64(div(fileNumber, validationSize))
        edepNumber = filter(x->occursin(fileKey, x), readdir())[1+((fileNumber-1) - Int(div((fileNumber-1), validationSize)))+r][16:18]
        tumorNumber = filter(x->occursin("tumor", x), readdir())[1+r][16:18]
        for i = 1:angles
        	maxVal = maximum(readdlm(string("processed_edep_", edepNumber, ".txt")))
        	minVal = minimum(readdlm(string("processed_edep_", edepNumber, ".txt")))
        	temp = ((transpose(reshape(readdlm(string("processed_edep_", edepNumber, ".txt"))[Int64(i*60/angles),:], (28,28)).-minVal)*255/maxVal))
        	contrast(temp)
            dataArrayCPU[:,:,i,r] = temp
            #dataArray[:,:,i,1,r+Int64(div(fileNumber, 1.25))] = Int16.(round.(flipInputData(transpose(reshape(readdlm(string("processed_edep_", tumorNumber, ".txt"))[Int64(i*60/angles),:], (28,28))))))
        end
        maxVal = maximum(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")))
        minVal = minimum(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")))
        #targetArrayCPU[:, r] = (transpose(reshape(dataArrayCPU[:,:,:,r], (28*28*angles, 1)))) 
        targetArrayCPU[:,r] = ((transpose(reshape(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")), (21952,1)).-minVal)*255/maxVal))
        #targetArray[:,r+Int64(div(fileNumber, 1.25))] = Int16.(round.(flipTargetData(transpose(reshape(readdlm(string("tumor_28_28_28_", tumorNumber, "_montage.txt")), (21952,1))))*255/maxVal))
        print("Training data: ", string("processed_edep_", edepNumber, ".txt"), "/", string("tumor_28_28_28_", edepNumber, "_montage.txt"), '\n')
    end
    
    dataArrayCPU[:,:,:,size(dataArrayCPU)[4]] = testArrayCPU[:,:,:,1]
    targetArrayCPU[:,:,:,size(targetArrayCPU)[4]] = testTargetArrayCPU[:,:,:,1]

    for r = 1:((fileNumber-1) - Int(div((fileNumber-1), validationSize)))
        edepNumber = filter(x->occursin(fileKey, x), readdir())[1+r][16:18]
        tumorNumber = filter(x->occursin("tumor", x), readdir())[1+Int64(div(fileNumber, validationSize))+r][16:18]
        for i = 1:angles
            maxVal = maximum(readdlm(string("processed_edep_", edepNumber, ".txt")))
            minVal = minimum(readdlm(string("processed_edep_", edepNumber, ".txt")))
            temp = ((transpose(reshape(readdlm(string("processed_edep_", edepNumber, ".txt"))[Int64(i*60/angles),:], (28,28)).-minVal)*255/maxVal))
            contrast(temp)
            validationArrayCPU[:,:,i,r] = temp
            #dataArray[:,:,i,1,r+Int64(div(fileNumber, 1.25))] = Int16.(round.(flipInputData(transpose(reshape(readdlm(string("processed_edep_", tumorNumber, ".txt"))[Int64(i*60/angles),:], (28,28))))))
        end
        maxVal = maximum(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")))
        minVal = minimum(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")))
        #validationTargetArrayCPU[:, r] = (transpose(reshape(dataArrayCPU[:,:,:,r], (28*28*angles, 1)))) 
        validationTargetArrayCPU[:,r] = ((transpose(reshape(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")), (21952,1)).-minVal)*255/maxVal))
        #targetArrayCPU[:,r] = ((transpose(reshape(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")), (21952,1)).-minVal)*255/maxVal))
        #targetArray[:,r+Int64(div(fileNumber, 1.25))] = Int16.(round.(flipTargetData(transpose(reshape(readdlm(string("tumor_28_28_28_", tumorNumber, "_montage.txt")), (21952,1))))*255/maxVal))
    end

    @show typeof(dataArrayCPU)
 
    
    validationArray = validationArrayCPU[:,:,:,:]
    validationTargetArray = validationTargetArrayCPU[:,:]
    testArray = testArrayCPU
    testTargetArray = testTargetArrayCPU
    dataArray = dataArrayCPU
    targetArray = targetArrayCPU

    
    #CUDA.reclaim()
    #CUDA.memory_status() 
    
    print(size(validationArray), "\n")
    print(size(validationTargetArray), "\n")
    print(size(dataArray), "\n")
    
    @show validationArray[:,:,:,1]
    @show validationArray[:,:,:,size(validationArray)[4]]
    @show dataArray[:,:,:,1]
    @show dataArray[:,:,:,size(dataArray)[4]]
    
    #CUDA.memory_status()

    #@show typeof(dataArray)

    print(string("Grabbed training files ", "\n"))

    if algorithm == "default"
        m = Chain(Conv((3,3), angles=>16*angles, pad=(1,1), relu),
        x -> maxpool(x, (2,2)),
        
        Conv((2,2), 16*angles=>32*angles, pad=(1,1), relu),
        x -> maxpool(x, (2,2)),   
        
        Conv((1,1), 32*angles=>64*angles, pad=(1,1), relu), 
        x -> maxpool(x, (2,2)),  
        
        x -> reshape(x, :, size(x,4)), 
        Dense(angles*1024, 28*28*28))

        print("Running default algorithm.", "\n")
    else
        m = algorithm
        print("Running costom algorithm.", "\n")
    end
    @show typeof(m)

    #CUDA.memory_status()
 
    print("grabbed algo", '\n')
    #loss(x, y) = logitcrossentropy(abs.(m(x)), y) #loss function is an exponential difference of each value in arrays
    loss(x, y) = mean((m(x)-y).^2) #loss function is an exponential difference of each value in arrays
    
    #print(size(reshape(validationArray, (28*28*angles, 37))), '\t', size(validationTargetArray), '\n')
    
    #print(size(m(validationArray)), '\t', size(validationTargetArray), '\n')
    print(minimum(m(dataArray)), '\t', maximum(m(dataArray)), '\t', m(dataArray)[1:5], '\t', targetArray[1:5], '\n')
    print(minimum(m(validationArray)), '\t', maximum(m(validationArray)), '\t', m(validationArray)[1:5], '\t', validationTargetArray[1:5], '\n')
    print(minimum(m(testArray)), '\t', maximum(m(testArray)), '\t', m(testArray)[1:5], '\t', testTargetArray[1:5], '\n')
    
    
    #loss(x, y) = mean(abs.((reshape(x, (28*28*angles, 37))-y).^2)) + 1
    
    print("created loss function", '\n')

    loss(validationArray, validationTargetArray)
 
   	validationValue = 10000
    R2 = r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(testArray)[:,1]), B=testTargetArray[:,1])))

    opt = ADAM()
    plottedArray = zeros(0)
    highR = 0.0
    lowVal = validationValue
    runGraphingNumber = lpad(size(filter(x->occursin("graphingDataR2", x), readdir()))[1] + 1, 3, "0")
    parseSize = size(dataArrayCPU)[4]

    for i = 1:runNumber
        for n = 1:div(size(dataArrayCPU)[4],parseSize)
        	print("-----------------------------------", '\n', '\n')
            print("Running files ", (n-1)*parseSize+1, "-", (n-1)*parseSize+parseSize, "\n")
            
            #print("Loss Values: ", mean((m(dataArray) - targetArray).^2), '\t', mean((m(validationArray) - validationTargetArray).^2), '\n')
            print("Loss Values: ", loss(dataArray, targetArray), '\t', loss(validationArray, validationTargetArray), '\t', loss(testArray, testTargetArray), '\n')

            dataset = (repeated((dataArray, targetArray), 200))

            evalcb = () -> @show (loss(dataArray, targetArray)) #Evaluation on call back shows loss function for that run
            
            global mFinal = m
            validationValue = loss(validationArray, validationTargetArray)
            R2 = r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(testArray)[:,1]), B=testTargetArray[:,1])))
            print("Training data . . .", "\n")
            
            Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10))  #train data set , cb = throttle(evalcb, 10)
            
            print(string("Previous validation value: ", validationValue, '\n'))
            print(string("New validation value: ", loss(validationArray, validationTargetArray), '\n', '\n'))
            
            print(minimum(m(dataArray)), '\t', maximum(m(dataArray)), '\t', m(dataArray)[1:5], '\t', targetArray[1:5], '\n')
            print(minimum(m(validationArray)), '\t', maximum(m(validationArray)), '\t', m(validationArray)[1:5], '\t', validationTargetArray[1:5], '\n')
            print(minimum(m(testArray)), '\t', maximum(m(testArray)), '\t', m(testArray)[1:5], '\t', testTargetArray[1:5], '\n')
    
            print(string("Previous R2 value: ", R2, '\n'))
    using Flux, Statistics
using Flux: crossentropy, throttle, logitcrossentropy
using Base.Iterators: repeated, partition, flatten
using DelimitedFiles
using LinearAlgebra
using GLM
using RDatasets
using BSON: @save, @load
using Random
using Base.Threads 
using CUDA

function runNNfiles(args)
    runNumber = parse(Int64, args[1])
    algorithm = args[2]
    angles = parse(Int64, args[3])
    folder = "/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/edep_extraction_data/processed_tumors_28_28_28"
    fileKey = "processed_edep_"
    validationSize = 1.05
    
    #CUDA.memory_status()
    
    lightValue = 128
    contrast = x -> for i = 1:size(x)[1] 
           for r = 1:size(x)[2]
               if x[i,r] > lightValue
                   x[i,r] = (sqrt(x[i,r]/lightValue) * sqrt(255/lightValue)) * lightValue
               else
                   x[i,r] = (x[i,r]/lightValue)^2*lightValue
               end
           end
       end


    cd(folder) #Move directory to directed folder

    pixelNumber = size(split.(Iterators.take(eachline(filter(x->occursin(fileKey, x), readdir())[1]), 1), '\t')[1])[1]
    fileNumber = size(filter(x->occursin(fileKey, x), readdir()))[1] - 1

    print(string("pixels: ", pixelNumber, '\n', "fileNumber: ", fileNumber, '\n'))
     
    dataArrayCPU = (Array{Float32}(undef, Int16(sqrt(pixelNumber)), Int16(sqrt(pixelNumber)), angles, 1+(Int16(div(fileNumber, validationSize)))))
    targetArrayCPU = (Array{Float32}(undef, Int16(784^(3/2)), 1+(Int16(div(fileNumber, validationSize)))))
    #targetArrayCPU = (Array{Float32}(undef, Int32(28*28*angles), (Int16(div(fileNumber, validationSize)))))
    validationArrayCPU = (Array{Float32}(undef, Int16(sqrt(pixelNumber)), Int16(sqrt(pixelNumber)), angles, (fileNumber) - Int16(div((fileNumber), validationSize))))
    validationTargetArrayCPU = (Array{Float32}(undef, Int16(784^(3/2)), (fileNumber) - Int16(div((fileNumber), validationSize))))
    #validationTargetArrayCPU = (Array{Float32}(undef, Int32(28*28*angles), (fileNumber) - Int16(div((fileNumber), validationSize))))
    testArrayCPU = (Array{Float32}(undef, Int16(sqrt(pixelNumber)), Int16(sqrt(pixelNumber)), Int16(angles), 1))
    testTargetArrayCPU = (Array{Float32}(undef, Int16(784^(3/2)), 1))
    #testTargetArrayCPU = (Array{Float32}(undef, Int32(28*28*angles), 1))
    
    @show typeof(dataArrayCPU)

    for i = 1:angles
        maxVal = maximum(readdlm(filter(x->occursin("processed", x), readdir())[1]))
        minVal = minimum(readdlm(filter(x->occursin("processed", x), readdir())[1]))
        temp = (transpose(reshape(readdlm(filter(x->occursin("processed", x), readdir())[1])[Int64(i*60/angles),:], (28,28)).-minVal)*255/maxVal)
        contrast(temp)
        testArrayCPU[:,:,i,1] = temp
    end
    edepNumber = filter(x->occursin(fileKey, x), readdir())[1][16:18]
    tumorNumber = filter(x->occursin("tumor", x), readdir())[1][16:18]
    maxVal = maximum(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")))
    minVal = minimum(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")))
    #testTargetArrayCPU[:,1] = (transpose(reshape(testArrayCPU[:,:,:,1], (28*28*angles, 1)))) 
    testTargetArrayCPU[:,1] = ((transpose(reshape(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")), (21952,1)).-minVal)*255/maxVal))
    print("Test data: ", string("processed_edep_", edepNumber, ".txt"), "/", string("tumor_28_28_28_", edepNumber, "_montage.txt"), '\n')


    for r = 1:Int64(div(fileNumber, validationSize))
        edepNumber = filter(x->occursin(fileKey, x), readdir())[1+((fileNumber-1) - Int(div((fileNumber-1), validationSize)))+r][16:18]
        tumorNumber = filter(x->occursin("tumor", x), readdir())[1+r][16:18]
        for i = 1:angles
        	maxVal = maximum(readdlm(string("processed_edep_", edepNumber, ".txt")))
        	minVal = minimum(readdlm(string("processed_edep_", edepNumber, ".txt")))
        	temp = ((transpose(reshape(readdlm(string("processed_edep_", edepNumber, ".txt"))[Int64(i*60/angles),:], (28,28)).-minVal)*255/maxVal))
        	contrast(temp)
            dataArrayCPU[:,:,i,r] = temp
            #dataArray[:,:,i,1,r+Int64(div(fileNumber, 1.25))] = Int16.(round.(flipInputData(transpose(reshape(readdlm(string("processed_edep_", tumorNumber, ".txt"))[Int64(i*60/angles),:], (28,28))))))
        end
        maxVal = maximum(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")))
        minVal = minimum(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")))
        #targetArrayCPU[:, r] = (transpose(reshape(dataArrayCPU[:,:,:,r], (28*28*angles, 1)))) 
        targetArrayCPU[:,r] = ((transpose(reshape(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")), (21952,1)).-minVal)*255/maxVal))
        #targetArray[:,r+Int64(div(fileNumber, 1.25))] = Int16.(round.(flipTargetData(transpose(reshape(readdlm(string("tumor_28_28_28_", tumorNumber, "_montage.txt")), (21952,1))))*255/maxVal))
        print("Training data: ", string("processed_edep_", edepNumber, ".txt"), "/", string("tumor_28_28_28_", edepNumber, "_montage.txt"), '\n')
    end
    
    dataArrayCPU[:,:,:,size(dataArrayCPU)[4]] = testArrayCPU[:,:,:,1]
    targetArrayCPU[:,:,:,size(targetArrayCPU)[4]] = testTargetArrayCPU[:,:,:,1]

    for r = 1:((fileNumber-1) - Int(div((fileNumber-1), validationSize)))
        edepNumber = filter(x->occursin(fileKey, x), readdir())[1+r][16:18]
        tumorNumber = filter(x->occursin("tumor", x), readdir())[1+Int64(div(fileNumber, validationSize))+r][16:18]
        for i = 1:angles
        	maxVal = maximum(readdlm(string("processed_edep_", edepNumber, ".txt")))
        	minVal = minimum(readdlm(string("processed_edep_", edepNumber, ".txt")))
        	temp = ((transpose(reshape(readdlm(string("processed_edep_", edepNumber, ".txt"))[Int64(i*60/angles),:], (28,28)).-minVal)*255/maxVal))
        	contrast(temp)
            validationArrayCPU[:,:,i,r] = temp
            #dataArray[:,:,i,1,r+Int64(div(fileNumber, 1.25))] = Int16.(round.(flipInputData(transpose(reshape(readdlm(string("processed_edep_", tumorNumber, ".txt"))[Int64(i*60/angles),:], (28,28))))))
        end
        maxVal = maximum(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")))
        minVal = minimum(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")))
        #validationTargetArrayCPU[:, r] = (transpose(reshape(dataArrayCPU[:,:,:,r], (28*28*angles, 1)))) 
        validationTargetArrayCPU[:,r] = ((transpose(reshape(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")), (21952,1)).-minVal)*255/maxVal))
        #targetArrayCPU[:,r] = ((transpose(reshape(readdlm(string("tumor_28_28_28_", edepNumber, "_montage.txt")), (21952,1)).-minVal)*255/maxVal))
        #targetArray[:,r+Int64(div(fileNumber, 1.25))] = Int16.(round.(flipTargetData(transpose(reshape(readdlm(string("tumor_28_28_28_", tumorNumber, "_montage.txt")), (21952,1))))*255/maxVal))
    end

    @show typeof(dataArrayCPU)
 
    
    validationArray = validationArrayCPU[:,:,:,:]
    validationTargetArray = validationTargetArrayCPU[:,:]
    testArray = testArrayCPU
    testTargetArray = testTargetArrayCPU
    dataArray = dataArrayCPU
    targetArray = targetArrayCPU

    
    #CUDA.reclaim()
    #CUDA.memory_status() 
    
    print(size(validationArray), "\n")
    print(size(validationTargetArray), "\n")
    print(size(dataArray), "\n")
    
    @show validationArray[:,:,:,1]
    @show validationArray[:,:,:,size(validationArray)[4]]
    @show dataArray[:,:,:,1]
    @show dataArray[:,:,:,size(dataArray)[4]]
    
    #CUDA.memory_status()

    #@show typeof(dataArray)

    print(string("Grabbed training files ", "\n"))

    if algorithm == "default"
        m = Chain(Conv((3,3), angles=>16*angles, pad=(1,1), relu),
        x -> maxpool(x, (2,2)),
        
        Conv((2,2), 16*angles=>32*angles, pad=(1,1), relu),
        x -> maxpool(x, (2,2)),   
        
        Conv((1,1), 32*angles=>64*angles, pad=(1,1), relu), 
        x -> maxpool(x, (2,2)),  
        
        x -> reshape(x, :, size(x,4)), 
        Dense(angles*1024, 28*28*28))

        print("Running default algorithm.", "\n")
    else
        m = algorithm
        print("Running costom algorithm.", "\n")
    end
    @show typeof(m)

    #CUDA.memory_status()
 
    print("grabbed algo", '\n')
    #loss(x, y) = logitcrossentropy(abs.(m(x)), y) #loss function is an exponential difference of each value in arrays
    loss(x, y) = mean((m(x)-y).^2) #loss function is an exponential difference of each value in arrays
    
    #print(size(reshape(validationArray, (28*28*angles, 37))), '\t', size(validationTargetArray), '\n')
    
    #print(size(m(validationArray)), '\t', size(validationTargetArray), '\n')
    print(minimum(m(dataArray)), '\t', maximum(m(dataArray)), '\t', m(dataArray)[1:5], '\t', targetArray[1:5], '\n')
    print(minimum(m(validationArray)), '\t', maximum(m(validationArray)), '\t', m(validationArray)[1:5], '\t', validationTargetArray[1:5], '\n')
    print(minimum(m(testArray)), '\t', maximum(m(testArray)), '\t', m(testArray)[1:5], '\t', testTargetArray[1:5], '\n')
    
    
    #loss(x, y) = mean(abs.((reshape(x, (28*28*angles, 37))-y).^2)) + 1
    
    print("created loss function", '\n')

    loss(validationArray, validationTargetArray)
 
   	validationValue = 10000
    R2 = r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(testArray)[:,1]), B=testTargetArray[:,1])))

    opt = ADAM()
    plottedArray = zeros(0)
    highR = 0.0
    lowVal = validationValue
    runGraphingNumber = lpad(size(filter(x->occursin("graphingDataR2", x), readdir()))[1] + 1, 3, "0")
    parseSize = size(dataArrayCPU)[4]

    for i = 1:runNumber
        for n = 1:div(size(dataArrayCPU)[4],parseSize)
        	print("-----------------------------------", '\n', '\n')
            print("Running files ", (n-1)*parseSize+1, "-", (n-1)*parseSize+parseSize, "\n")
            
            #print("Loss Values: ", mean((m(dataArray) - targetArray).^2), '\t', mean((m(validationArray) - validationTargetArray).^2), '\n')
            print("Loss Values: ", loss(dataArray, targetArray), '\t', loss(validationArray, validationTargetArray), '\t', loss(testArray, testTargetArray), '\n')

            dataset = (repeated((dataArray, targetArray), 200))

            evalcb = () -> @show (loss(dataArray, targetArray)) #Evaluation on call back shows loss function for that run
            
            global mFinal = m
            validationValue = loss(validationArray, validationTargetArray)
            R2 = r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(testArray)[:,1]), B=testTargetArray[:,1])))
            print("Training data . . .", "\n")
            
            Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10))  #train data set , cb = throttle(evalcb, 10)
            
            print(string("Previous validation value: ", validationValue, '\n'))
	    	print(string("New validation value: ", loss(validationArray, validationTargetArray), '\n', '\n'))
	    	
	    	print(minimum(m(dataArray)), '\t', maximum(m(dataArray)), '\t', m(dataArray)[1:5], '\t', targetArray[1:5], '\n')
    		print(minimum(m(validationArray)), '\t', maximum(m(validationArray)), '\t', m(validationArray)[1:5], '\t', validationTargetArray[1:5], '\n')
    		print(minimum(m(testArray)), '\t', maximum(m(testArray)), '\t', m(testArray)[1:5], '\t', testTargetArray[1:5], '\n')
    		
	    	print(string("Previous R2 value: ", R2, '\n'))
            print("R2 value: ", r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(testArray)[:,1]), B=testTargetArray[:,1]))), "\n")
	
	    	#append!(plottedArray, loss(validationArray, validationTargetArray))
	    	#print(plottedArray, "\n")
	    
	    	cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development/juliaResults/juliaGraphingResults")
	    	io = open(string("graphingDataR2", runGraphingNumber, ".txt"), "w");
	    	write(io, string(R2, ", "))
	    	close(io)
	    	io = open(string("graphingDataVal", runGraphingNumber, ".txt"), "w");
	    	write(io, string(validationValue, ", "))
	    	close(io)
	    	cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development")
	    
	    	print("Run number: ", i, "\n")
    	end
    	
    	if (loss(validationArray, validationTargetArray) < lowVal)
			print("New Low Validation!", "\n")
			lowVal = validationValue
			cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development/juliaResults")

			normalize = x -> (x)*(255/maximum(x))

			writedlm(string("target_deposition.txt"), Transpose(reshape(normalize(testTargetArray)[:,1], (784, 28))), " ")
			writedlm(string("test_deposition_val.txt"), Transpose(reshape(normalize(m(testArray))[:,1], (784, 28))), " ")
			
			cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development")
	
	   	end
	
	   	if (R2 > highR)                 
           	print("New High R2!", "\n")
            highR = R2 
            cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development/juliaResults")

            normalize = x -> (x)*(255/maximum(x))

            writedlm(string("target_deposition.txt"), Transpose(reshape(normalize(testTargetArray)[:,1], (28*28, 28))), " ")
            writedlm(string("test_deposition_r2.txt"), Transpose(reshape(normalize(m(testArray))[:,1], (28*28, 28))), " ")
         

            cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development")

         end
    end
    
    print("training terminated!", '\n')   
    highR = R2 
    cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development/juliaResults")

    normalize = x -> (x)*(255/maximum(x))
    writedlm(string("test_deposition_final.txt"), Transpose(reshape(normalize(m(testArray))[:,1], (28*28, 28))), " ") 
    writedlm(string("target_data_deposition_final.txt"), Transpose(reshape(normalize(targetArray)[:,size(dataArray)[4]], (28*28, 28))), " ")
    writedlm(string("test_data_deposition_final.txt"), Transpose(reshape(normalize(m(dataArray))[:,size(dataArray)[4]], (28*28, 28))), " ")
    
    cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development")

 #   cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/juliaResults")

 #   normalize = x -> (cpu(x))*(255/maximum(cpu(x)))

 #   for i = 1:size(testTargetArray)[2]
 #       writedlm(string("target_deposition_", i, ".txt"), Transpose(reshape(normalize(testTargetArray)[:,i], (784, 28))), " ")
 #       writedlm(string("test_deposition_", i, ".txt"), Transpose(reshape(normalize(mFinal(testArray))[:,i], (784, 28))), " ")
 #   end

 #    cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development")

    return mFinal, plottedArray
end

runNNfiles(ARGS)        print("R2 value: ", r2(lm(@formula(A  ~ B), DataFrame(A=Float64.(m(testArray)[:,1]), B=testTargetArray[:,1]))), "\n")
	
	    	#append!(plottedArray, loss(validationArray, validationTargetArray))
	    	#print(plottedArray, "\n")
	    
	    	cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development/juliaResults/juliaGraphingResults")
	    	io = open(string("graphingDataR2", runGraphingNumber, ".txt"), "w");
	    	write(io, string(R2, ", "))
	    	close(io)
	    	io = open(string("graphingDataVal", runGraphingNumber, ".txt"), "w");
	    	write(io, string(validationValue, ", "))
	    	close(io)
	    	cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development")
	    
	    	print("Run number: ", i, "\n")
    	end
    	
    	if (loss(validationArray, validationTargetArray) < lowVal)
			print("New Low Validation!", "\n")
			lowVal = validationValue
			cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development/juliaResults")

			normalize = x -> (x)*(255/maximum(x))

			writedlm(string("target_deposition.txt"), Transpose(reshape(normalize(testTargetArray)[:,1], (784, 28))), " ")
			writedlm(string("test_deposition_val.txt"), Transpose(reshape(normalize(m(testArray))[:,1], (784, 28))), " ")
			
			cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development")
	
	   	end
	
	   	if (R2 > highR)                 
           	print("New High R2!", "\n")
            highR = R2 
            cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development/juliaResults")

            normalize = x -> (x)*(255/maximum(x))

            writedlm(string("target_deposition.txt"), Transpose(reshape(normalize(testTargetArray)[:,1], (28*28, 28))), " ")
            writedlm(string("test_deposition_r2.txt"), Transpose(reshape(normalize(m(testArray))[:,1], (28*28, 28))), " ")
         

            cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development")

         end
    end
    
    print("training terminated!", '\n')   
    highR = R2 
    cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development/juliaResults")

    normalize = x -> (x)*(255/maximum(x))
    writedlm(string("test_deposition_final.txt"), Transpose(reshape(normalize(m(testArray))[:,1], (28*28, 28))), " ") 
    writedlm(string("target_data_deposition_final.txt"), Transpose(reshape(normalize(targetArray)[:,size(dataArray)[4]], (28*28, 28))), " ")
    writedlm(string("test_data_deposition_final.txt"), Transpose(reshape(normalize(m(dataArray))[:,size(dataArray)[4]], (28*28, 28))), " ")
    
    cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development")

 #   cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/juliaResults")

 #   normalize = x -> (cpu(x))*(255/maximum(cpu(x)))

 #   for i = 1:size(testTargetArray)[2]
 #       writedlm(string("target_deposition_", i, ".txt"), Transpose(reshape(normalize(testTargetArray)[:,i], (784, 28))), " ")
 #       writedlm(string("test_deposition_", i, ".txt"), Transpose(reshape(normalize(mFinal(testArray))[:,i], (784, 28))), " ")
 #   end

 #    cd("/home/gmf/Projects/Summer_2020_Proton_Imager/Proton_Imager/Neural_Net_Development")

    return mFinal, plottedArray
end

runNNfiles(ARGS)